# Accessing Data From The US Census

This notebook contains code to access data from the US Census' API. There are numerous datasets available ([*link*](https://www.census.gov/data/developers/data-sets.html)). The **American Community Survey 5-Year Data (2009-2019)** (ACS, [*link*](https://www.census.gov/data/developers/data-sets/acs-5year.html)) contains various demographic data collected across the country. This survey was selected in lieu of the 1-Year version as ZIP Codes are only supported at the five year level.

The schedule for the release of the data is located [here](https://www.census.gov/programs-surveys/acs/news/data-releases/2020/release-schedule.html). As of August 8, 2021 the 2019 ACS was the most recent release, which does not account for the effects of the lockdowns. The 2020 survey is planned to be released on March 17, 2022.

The [examples](https://api.census.gov/data/2019/acs/acs5/examples.html) page contains the formatting necessary to make calls and lists the geographic area of data that is available.